In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
zinc_raw_df = pd.read_csv('ZINC_data_5HT2A.csv', index_col=0)
chembl_raw_df = pd.read_csv('Chembl_data_5HT2A.csv', index_col=0)

In [5]:
mask = chembl_raw_df['relation_clean'] == '='
chembl_raw_df.loc[mask, 'pKi_recalculated'] = -np.log10(chembl_raw_df.loc[mask, 'Standard Value'] / 1e9)
chembl_raw_df['pKi_delta'] = chembl_raw_df['pKi_recalculated'] - chembl_raw_df['pKi_numeric']
#zinc_raw_df['label'] = 'zinc'
#chembl_raw_df['label'] = 'chembl'
zinc_subset = zinc_raw_df[['pKi_numeric', 'smiles']].rename(columns={'smiles': 'Smiles', 'pKi_numeric': 'pKi'})
chembl_subset = chembl_raw_df.query("relation_clean == '='")[['pKi_recalculated', 'Smiles']].rename(columns={'pKi_recalculated': 'pKi'})

merged_data = pd.concat([zinc_subset, chembl_subset], ignore_index=True)
merged_data = merged_data.reset_index(drop=False)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6832 entries, 0 to 6831
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   6832 non-null   int64  
 1   pKi     6832 non-null   float64
 2   Smiles  6832 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 160.2+ KB


In [ ]:
merged_data = merged_data.reset_index(drop=False)

def deduplicate_smiles(smile):
    if len(smile) == 1:
        return smile
    delta = smile['pKi'].max() - smile['pKi'].min()
    if delta > 0.1:
        return pd.DataFrame(columns=smile.columns)
    else:
        idx = smile['index'].idxmin()
        return smile.loc[[idx]]


deduplicated_df = merged_data.groupby('Smiles', group_keys=False).apply(deduplicate_smiles).drop(columns='index')

In [6]:
def deduplicate_smiles(smile):
    if len(smile) == 1:
        return smile
    delta = smile['pKi'].max() - smile['pKi'].min()
    if delta > 0.1:
        return pd.DataFrame(columns=smile.columns)
    else:
        idx = smile['index'].idxmin()
        return smile.loc[[idx]]


deduplicated_df = merged_data.groupby('Smiles', group_keys=False).apply(deduplicate_smiles).drop(columns='index')
deduplicated_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4846 entries, 6291 to 1361
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pKi     4846 non-null   float64
 1   Smiles  4846 non-null   object 
dtypes: float64(1), object(1)
memory usage: 113.6+ KB


In [9]:
mordred_df = pd.read_csv('Mordred_descriptors_database.csv')
mordred_df.head()

C:\Users\User\anaconda3\envs\PythonforTensorflow\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,54,55,56,57,58,59,60,61,62,136,137,138,139,140,141,142,145,146,147,148,149,150,151,153,154,155,156,157,158,159,160,163,164,165,166,167,168,169,172,173,174,175,176,177,178,181,182,183,184,185,186,187,190,191,192,193,194,195,196,199,200,201,202,203,204,205,208,209,210,211,212,213,214,217,218,219,220,221,222,223,226,227,228,229,230,231,232,234,235,236,237,238,239,240,241,242,261,262,263,264,265,266,267,268,269,342,343,344,345,346,347,348,349,352,353,354,355,356,357,358,361,362,363,364,365,366,367,369,370,371,372,373,374,375,376,379,380,381,382,383,384,385,388,389,390,391,392,393,394,397,398,399,400,401,402,403,406,407,408,409,410,411,412,415,416,417,418,419,420,421,424,425,426,427,428,429,430,433,434,435,436,437,438,439,442,443,444,445,446,447,448,450,451,452,453,454,455,456,458,459,460,461,462,463,464,466,467,468,469,470,471,472,4

smiles  \
0  CC(=O)c1c(OCCCCN2CCN(c3cccc(F)c3)CC2)ccc2c(C)c...   
1                        COc1c2c(c(CCN)c3c1OCC3)CCO2   
2       COc1ccc2[nH]cc(CCNCc3ccc(-c4cn[nH]c4)o3)c2c1   
3       Oc1ccc(-c2ccc(CNCCc3c[nH]c4cc(F)ccc34)o2)cc1   
4  O=C(NC/C=C/CN1CCN(c2cccc(Cl)c2Cl)CC1)c1ccc(-c2...   

                                                 ABC  \
0  module 'numpy' has no attribute 'float'.\n`np....   
1  module 'numpy' has no attribute 'float'.\n`np....   
2  module 'numpy' has no attribute 'float'.\n`np....   
3  module 'numpy' has no attribute 'float'.\n`np....   
4  module 'numpy' has no attribute 'float'.\n`np....   

                                               ABCGG  nAcid  nBase  \
0  module 'numpy' has no attribute 'float'.\n`np....      0      1   
1  module 'numpy' has no attribute 'float'.\n`np....      0      1   
2  module 'numpy' has no attribute 'float'.\n`np....      0      1   
3  module 'numpy' has no attribute 'float'.\n`np....      0      1   
4  module 'numpy' has no attribute 'float'.\n`np....      0      1   

              SpAbs_A             SpMax_A           SpDiam_A  \
0   42.06466278808458  2.4743940799154185  4.948788159830838   
1  22.566899740452094  2.5393429456080714  4.980643062207491   
2   33.40302700153903    2.42142639341503  4.721184948178399   
3  34.277891020379556  2.4149865997702755  4.699237272854239   
4   43.73290574033282   2.375216679151583  4.750433358303166   

               SpAD_A             SpMAD_A  ...      SRW10     TSRW10  \
0   42.06466278808458  1.2746867511540783  ...  10.410697  69.176088   
1  22.566899740452094   1.327464690614829  ...   9.956033  66.014285   
2   33.40302700153903  1.3361210800615613  ...   9.992277  76.384627   
3   34.27789102037955   1.318380423860752  ...  10.067263  76.660405   
4   43.73290574033282  1.3252395678888733  ...  10.267193  68.855801   

           MW       AMW WPath WPol Zagreb1  Zagreb2            mZagreb1  \
0  452.211136  7.293728  3940   53   172.0    201.0  10.472222222222221   
1  235.120843  6.915319   452   28    92.0    114.0   4.916666666666667   
2  336.158626  7.470192  1815   33   132.0    155.0   6.027777777777779   
3  350.143056  7.780957  2065   36   138.0    161.0   6.888888888888889   
4  480.148367  8.138108  4316   50   168.0    194.0                9.25   

   mZagreb2  
0  7.250000  
1  3.916667  
2  5.583333  
3  5.666667  
4  7.388889  

[5 rows x 1614 columns]

In [10]:
# int64 lub float64
numeric_cols = mordred_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# smiles object
if 'smiles' in mordred_df.columns and mordred_df['smiles'].dtype == 'object':
    selected_cols = ['smiles'] + numeric_cols
else:
    selected_cols = numeric_cols
# nowy df
mordred_df_cleaned = mordred_df[selected_cols]
mordred_df_cleaned.info()
mordred_df_cleaned.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009 entries, 0 to 7008
Columns: 770 entries, smiles to mZagreb2
dtypes: float64(457), int64(312), object(1)
memory usage: 41.2+ MB


smiles  nAcid  nBase  nAromAtom  \
0  CC(=O)c1c(OCCCCN2CCN(c3cccc(F)c3)CC2)ccc2c(C)c...      0      1         16   
1                        COc1c2c(c(CCN)c3c1OCC3)CCO2      0      1          6   
2       COc1ccc2[nH]cc(CCNCc3ccc(-c4cn[nH]c4)o3)c2c1      0      1         19   
3       Oc1ccc(-c2ccc(CNCCc3c[nH]c4cc(F)ccc34)o2)cc1      0      1         20   
4  O=C(NC/C=C/CN1CCN(c2cccc(Cl)c2Cl)CC1)c1ccc(-c2...      0      1         18   

   nAromBond  nAtom  nHeavyAtom  nSpiro  nBridgehead  nHetero  ...     SRW09  \
0         17     62          33       0            0        7  ...  0.000000   
1          6     34          17       0            0        4  ...  7.321850   
2         20     45          25       0            0        6  ...  7.686621   
3         21     45          26       0            0        5  ...  7.401842   
4         18     59          33       0            0        7  ...  0.000000   

       SRW10     TSRW10          MW       AMW  WPath  WPol  Zagreb1  Zagreb2  \
0  10.410697  69.176088  452.211136  7.293728   3940    53    172.0    201.0   
1   9.956033  66.014285  235.120843  6.915319    452    28     92.0    114.0   
2   9.992277  76.384627  336.158626  7.470192   1815    33    132.0    155.0   
3  10.067263  76.660405  350.143056  7.780957   2065    36    138.0    161.0   
4  10.267193  68.855801  480.148367  8.138108   4316    50    168.0    194.0   

   mZagreb2  
0  7.250000  
1  3.916667  
2  5.583333  
3  5.666667  
4  7.388889  

[5 rows x 770 columns]

In [11]:
final_df = pd.merge(mordred_df_cleaned, deduplicated_df, left_on='smiles', right_on='Smiles', how='inner')
final_df = final_df.drop(columns=['smiles', 'Smiles'], axis=1)
final_df

nAcid  nBase  nAromAtom  nAromBond  nAtom  nHeavyAtom  nSpiro  \
0         0      1         16         17     62          33       0   
1         0      1         19         20     45          25       0   
2         0      1         20         21     45          26       0   
3         0      1         18         18     59          33       0   
4         0      5          6          6     48          24       1   
...     ...    ...        ...        ...    ...         ...     ...   
4841      0      2          6          6     75          36       0   
4842      0      1         15         16     46          26       0   
4843      0      2          6          6     75          36       0   
4844      0      2          6          6     53          23       1   
4845      0      2          6          6     53          23       1   

      nBridgehead  nHetero  nH  ...      SRW10     TSRW10          MW  \
0               0        7  29  ...  10.410697  69.176088  452.211136   
1               0        6  20  ...   9.992277  76.384627  336.158626   
2               0        5  19  ...  10.067263  76.660405  350.143056   
3               0        7  26  ...  10.267193  68.855801  480.148367   
4               0        7  24  ...  10.083849  58.556000  328.201159   
...           ...      ...  ..  ...        ...        ...         ...   
4841            2        7  39  ...  10.793865  87.904625  493.294057   
4842            0        9  20  ...  10.044944  74.369590  357.171336   
4843            2        7  39  ...  10.793865  87.904625  493.294057   
4844            0        4  30  ...  10.231387  57.718886  318.230728   
4845            0        4  30  ...  10.231387  57.718886  318.230728   

           AMW  WPath  WPol  Zagreb1  Zagreb2  mZagreb2       pKi  
0     7.293728   3940    53    172.0    201.0  7.250000  6.397940  
1     7.470192   1815    33    132.0    155.0  5.583333  7.552842  
2     7.780957   2065    36    138.0    161.0  5.666667  7.017729  
3     8.138108   4316    50    168.0    194.0  7.388889  7.200000  
4     6.837524   1409    36    124.0    143.0  5.361111  5.881735  
...        ...    ...   ...      ...      ...       ...       ...  
4841  6.577254   4674    63    198.0    240.0  7.763889  5.010000  
4842  7.764594   1862    36    136.0    157.0  5.722222  5.010000  
4843  6.577254   4674    63    198.0    240.0  7.763889  5.010000  
4844  6.004353   1110    40    120.0    144.0  5.347222  5.000000  
4845  6.004353   1110    40    120.0    144.0  5.347222  5.000000  

[4846 rows x 770 columns]

In [12]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn import tree

# 'pKi' to zmienna docelowa
X = final_df.drop(columns=['pKi'])
y = final_df['pKi']

# train+val oraz test (80% / 20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train+val na osobne train i val (75% / 25% z tych 80%)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# finalne proporcje:
# - train: 60%
# - val: 20%
# - test: 20%

print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

Train: (2907, 769), Validation: (969, 769), Test: (970, 769)


In [ ]:
import tensorflow as tf
import autokeras as ak
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Adding binar labels
y_train_bin = (y_train > 7).astype(int)
y_val_bin = (y_val > 7).astype(int)
y_test_bin = (y_test > 7).astype(int)

# Normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# train+val
X_full = np.vstack([X_train_scaled, X_val_scaled])
y_full = np.concatenate([y_train_bin, y_val_bin])

# AutoML
clf = ak.StructuredDataClassifier(
    max_trials=10,  
    overwrite=True,
    seed=42
)

clf.fit(X_full, y_full, epochs=30)

# 5. Prediction
y_pred = clf.predict(X_test_scaled).flatten()
y_pred_class = (y_pred > 0.5).astype(int)
best_model = clf.export_model()
best_model.save("best_autokeras_model.h5")

print(classification_report(y_test_bin, y_pred_class, target_names=["inactive", "active"]))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\PythonforTensorflow\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-8f834eb6ff48>", line 2, in <module>
    import autokeras as ak
ModuleNotFoundError: No module named 'autokeras'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\PythonforTensorflow\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\PythonforTensorflow\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_